In [2]:
import pandas as pd
import duckdb
import os
import os, json
from uuid import uuid4
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [3]:
conn = duckdb.connect('/srv/data/grela_v0-2.duckdb', read_only=True)

In [4]:
query = """
        SELECT s.grela_id,
               s.sentence_id,
               s.text,
               w.*
        FROM sentences s
                 JOIN works w ON s.grela_id = w.grela_id
        WHERE w.grela_id LIKE 'vulgate_%' \
        """

vulgate_sentences = conn.execute(query).fetchdf()

In [5]:
len(vulgate_sentences)

35254

In [7]:

query = """
        SELECT
            s.grela_id,
            s.sentence_id,
            s.text,
            w.*
        FROM sentences s
        JOIN works w ON s.grela_id = w.grela_id
        WHERE (w.not_before > 1000 AND w.not_before < 1100)
           OR (w.not_after > 1000 AND w.not_after < 1100);
"""

century11 = conn.execute(query).fetchdf()

In [8]:
century11

,grela_id,sentence_id,text,grela_source,grela_id_1,author,title,not_before,not_after,lagt_tlg_epithet,lagt_genre,lagt_provenience,noscemus_place,noscemus_genre,noscemus_discipline,title_short,emlap_noscemus_id,place_publication,place_geonames,author_viaf,title_viaf,date_random,token_count
0,cc_10949,cc_10949_0,De gemmis 1637-38 Gemmis a gummi nomen posuere...,cc,cc_10949,Marbodus Redonensis,De gemmis,1035.0,1123.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1038.0,6111
1,cc_10949,cc_10949_1,Nomine sed lapidis species signatur utroque;,cc,cc_10949,Marbodus Redonensis,De gemmis,1035.0,1123.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1038.0,6111
2,cc_10949,cc_10949_2,Propter quod #lapidum@# titulo liber iste nota...,cc,cc_10949,Marbodus Redonensis,De gemmis,1035.0,1123.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1038.0,6111
3,cc_10949,cc_10949_3,Prologus.,cc,cc_10949,Marbodus Redonensis,De gemmis,1035.0,1123.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1038.0,6111
4,cc_10949,cc_10949_4,"Euax rex Arabum legitur scripsisse Neroni, Qui...",cc,cc_10949,Marbodus Redonensis,De gemmis,1035.0,1123.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1038.0,6111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1357086,cc_10456,cc_10456_27,"Si uestrae igitur, summe Pater, dignitati plac...",cc,cc_10456,Raymundus Arnalli1001-1100,Epistola ad Bernardum,1001.0,1100.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1023.0,1022
1357087,cc_10456,cc_10456_28,Si enim quod exspecto apud uestram misericordi...,cc,cc_10456,Raymundus Arnalli1001-1100,Epistola ad Bernardum,1001.0,1100.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1023.0,1022
1357088,cc_10456,cc_10456_29,Interim autem usque ad festiuitatem sancti Mic...,cc,cc_10456,Raymundus Arnalli1001-1100,Epistola ad Bernardum,1001.0,1100.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1023.0,1022
1357089,cc_10456,cc_10456_30,Ualete.,cc,cc_10456,Raymundus Arnalli1001-1100,Epistola ad Bernardum,1001.0,1100.0,None,None,None,None,None,None,None,NaN,None,None,NaN,NaN,1023.0,1022


In [ ]:
len(century11)#%%
query = """
WITH subcorpora AS (
    SELECT
        SUBSTR(grela_id, 0, INSTR(grela_id, '_')) AS subcorpus,
        COUNT(DISTINCT grela_id) AS works_N,
        COUNT(DISTINCT sentence_id) AS sentences_N,
        COUNT(*) AS tokens_N
    FROM tokens
    GROUP BY subcorpus
)
SELECT * FROM subcorpora
ORDER BY subcorpus;
"""

# Execute the query and fetch as a pandas DataFrame
subcorpus_stats_df = conn.execute(query).fetchdf()

In [27]:
query = """
SELECT *
FROM tokens
WHERE grela_id LIKE 'vulgate_tlg0031%' OR grela_id LIKE 'vulgate_tlg0527%';
"""

result_df = conn.execute(query).fetchdf()

In [5]:
subcorpus_stats_df

,subcorpus,works_N,sentences_N,tokens_N
0,cc,7819,11835457,201939293
1,emlap,73,220846,3495212
2,lagt,1957,2703678,35808742
3,noscemus,996,11802783,139401899
4,vulgate,73,35254,603091


In [6]:
markdown_table = subcorpus_stats_df.set_index("subcorpus").applymap("{:,.0f}".format).to_markdown()
print(markdown_table)

| subcorpus   | works_N   | sentences_N   | tokens_N    |
|:------------|:----------|:--------------|:------------|
| cc          | 7,819     | 11,835,457    | 201,939,293 |
| emlap       | 73        | 220,846       | 3,495,212   |
| lagt        | 1,957     | 2,703,678     | 35,808,742  |
| noscemus    | 996       | 11,802,783    | 139,401,899 |
| vulgate     | 73        | 35,254        | 603,091     |


/tmp/ipykernel_54758/3358712856.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  markdown_table = subcorpus_stats_df.set_index("subcorpus").applymap("{:,.0f}".format).to_markdown()


In [15]:
def get_grouped_token_data_with_metadata(conn, lemma: str, pos_tags: list):
    """
    Return one row per matching token (lemma + POS filter) with:
      • full sentence token list
      • ±10-token concordance window
      • minimal work-level metadata
    """

    pos_placeholder = ", ".join(f"'{tag}'" for tag in pos_tags)

    query = f"""
    /*──────────────── 1. anchor tokens ────────────────*/
    WITH target_matches AS (
        SELECT
            t.sentence_id,
            t.grela_id,
            s.position                     AS sentence_position,
            t.token_id                     AS target_token_id,
            t.char_start                   AS target_char_start,
            t.char_end                     AS target_char_end
        FROM tokens   AS t
        JOIN sentences AS s USING (sentence_id)
        WHERE  t.lemma = ?
          AND  t.pos   IN ({pos_placeholder})
    ),

    /*──────────────── 2. ±1-sentence context ──────────*/
    context_3sents AS (
        SELECT
            tm.sentence_id,
            STRING_AGG(s.text, ' | ' ORDER BY s.position) AS context_3sents
        FROM target_matches tm
        JOIN sentences s
          ON s.grela_id = tm.grela_id
         AND s.position BETWEEN tm.sentence_position - 1
                            AND tm.sentence_position + 1
        GROUP BY tm.sentence_id
    ),

    /*──────────────── 3. full token list per sentence ─*/
    sentence_tokens AS (
        SELECT
            sentence_id,
            LIST(
              STRUCT_PACK(
                 token_id    := token_id,
                 token_text  := token_text,
                 lemma       := lemma,
                 pos         := pos,
                 char_start  := char_start,
                 char_end    := char_end,
                 sentence_id := sentence_id
              )
              ORDER BY token_id
            ) AS tokens
        FROM tokens
        WHERE sentence_id IN (SELECT DISTINCT sentence_id FROM target_matches)
        GROUP BY sentence_id
    ),

    /*──────────────── 4. ±10-token concordance window ─*/
    concordance_tokens AS (
        SELECT
            tm.sentence_id,
            tm.target_token_id,
            LIST(
              STRUCT_PACK(
                 token_id    := ct.token_id,
                 token_text  := ct.token_text,
                 lemma       := ct.lemma,
                 pos         := ct.pos,
                 char_start  := ct.char_start,
                 char_end    := ct.char_end,
                 sentence_id := ct.sentence_id
              )
              ORDER BY ct.token_id
            ) AS concordance_tokens
        FROM target_matches tm
        JOIN tokens        ct
          ON ct.grela_id = tm.grela_id
         AND ct.token_id BETWEEN tm.target_token_id - 10
                            AND tm.target_token_id + 10
        GROUP BY tm.sentence_id, tm.target_token_id
    )

    /*──────────────── 5. final projection (custom column order) ─────*/
    SELECT
        w.author,
        w.title,
        tm.grela_id AS grela_id,
        tm.sentence_id,
        s.text                    AS sentence_text,
        c3.context_3sents,

        st.tokens                AS tokens,
        ct.concordance_tokens    AS concordance_tokens,

        w.not_before,
        w.not_after,
        w.date_random,
        w.lagt_genre,
        w.lagt_provenience,
        w.noscemus_genre,
        w.noscemus_discipline,

        tm.target_token_id,
        tm.target_char_start,
        tm.target_char_end

    FROM target_matches      tm
    JOIN sentences           s   USING (sentence_id)
    JOIN works               w   ON w.grela_id = tm.grela_id
    LEFT JOIN context_3sents c3  USING (sentence_id)
    LEFT JOIN sentence_tokens st USING (sentence_id)
    LEFT JOIN concordance_tokens ct
           ON  ct.sentence_id    = tm.sentence_id
          AND ct.target_token_id = tm.target_token_id
    ORDER BY tm.sentence_id, tm.target_token_id;
    """

    return conn.execute(query, [lemma]).fetchdf()

In [16]:
result_df = get_grouped_token_data_with_metadata(conn, lemma="deus", pos_tags=["NOUN", "PROPN", "ADJ"])

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
len(result_df)

189258

In [18]:
result_df.sample(10)

,author,title,grela_id,sentence_id,sentence_text,context_3sents,tokens,concordance_tokens,not_before,not_after,date_random,lagt_genre,lagt_provenience,noscemus_genre,noscemus_discipline,target_token_id,target_char_start,target_char_end
150753,"Sorbière, Samuel",Ad virum illustrem Henricum Ludovicum Habertum...,noscemus_693174,noscemus_693174_18570,"nullam enim nouistis, nisi profectam a nem imm...","Praetereo autem quod habes Uoluptatum credo, n...","[{'token_id': 122102765, 'token_text': 'nullam...","[{'token_id': 122102785, 'token_text': ',', 'l...",1658.0,1658.0,1658.0,None,None,Biography,"Mathematics, Physics, Astronomy/Astrology/Cosm...",122102795,158,162
6914,Sicardus Cremonensis1155–1215,Chronicon,cc_11809,cc_11809_956,"#non est conueniens, ut homo judicet deos;","Lxxxi) @#. | #non est conueniens, ut homo judi...","[{'token_id': 266035572, 'token_text': '#', 'l...","[{'token_id': 266035570, 'token_text': '#', 'l...",1155.0,1215.0,1174.0,None,None,None,None,266035580,37,41
131092,Lactantius,Divinarum institutionum liber V,cc_6811,cc_6811_859,"Quid aliud dicam quam miseros, qui praedonum s...","nitunturque summa diligentia, ne qua sit anima...","[{'token_id': 298852647, 'token_text': 'Quid',...","[{'token_id': 298852650, 'token_text': 'quam',...",250.0,325.0,291.0,None,None,None,None,298852660,81,85
105032,Ioachim de Flore,Introduzione all'Apocalisse,cc_15557,cc_15557_175,[b.] Significata sunt hec omnia in octo illis ...,Septima uero pars de iudicio agit et de gloria...,"[{'token_id': 298000901, 'token_text': '[', 'l...","[{'token_id': 298000916, 'token_text': 'secund...",1132.0,1202.0,1161.0,None,None,None,None,298000926,134,137
6614,Alanus de Insulis,Contra haereticos,cc_11755,cc_11755_3580,"dii uero gentium alieni dii dicuntur, qui fals...","Nec est Deus alienus, qui uere est Deus; | Nec...","[{'token_id': 364744567, 'token_text': 'dii', ...","[{'token_id': 364744566, 'token_text': ';', 'l...",1128.0,1203.0,1149.0,None,None,None,None,364744576,48,51
73660,Augustinus Hipponensis,Epistulae 185-270 [CSEL],cc_14954,cc_14954_8016,"ingrati, scelesti, inreligiosi, sine affection...","inquit, se ipsos amantes, amatores pecuniae, e...","[{'token_id': 270227728, 'token_text': 'ingrat...","[{'token_id': 270227746, 'token_text': 'prodit...",354.0,430.0,390.0,None,None,None,None,270227756,167,170
35540,"Balde, Iacob",Poemata,cc_13993,cc_13993_20708,"Cum somnus, Deorum blandissimum munus, superfu...","mirabatur, si alius esset ab eo, quem ipse can...","[{'token_id': 353308466, 'token_text': 'Cum', ...","[{'token_id': 353308459, 'token_text': 'candid...",1604.0,1668.0,1665.0,None,None,None,None,353308469,12,18
28285,Giannozzo Manetti,De dignitate et excellentia hominis (I-II),cc_13026,cc_13026_383,"Qui quidem, cum per totum septimum et octauum ...","Sed, ut alia multa omittamus que a plerisque c...","[{'token_id': 222036668, 'token_text': 'Qui', ...","[{'token_id': 222036736, 'token_text': 'ut', '...",1396.0,1459.0,1446.0,None,None,None,None,222036746,466,469
105908,Benzo Albensis,Ad Heinricum IV imperatorem libri VII,cc_16653,cc_16653_3029,"Potest esse plasma dei, sol aut stella aliqua ...",Est autem in potestate liberi arbitrii Homines...,"[{'token_id': 374107227, 'token_text': 'Potest...","[{'token_id': 374107220, 'token_text': ',', 'l...",1010.0,1085.0,1033.0,None,None,None,None,374107230,19,22
117216,"Odo, Pietro",Carmina,cc_19839,cc_19839_431,"Ergo deo similem sese fecisse merendo, Qui pot...","numquam per uos errauerit ille, Per se quem nu...","[{'token_id': 380471196, 'token_text': 'Ergo',...","[{'token_id': 380471200, 'token_text': 'feciss...",NaN,NaN,NaN,None,None,None,None,380471210,74,77
